In [2]:
import os
from data_utils.base_dataset import BaseDataset

class OPV2VImageDataset(BaseDataset):
    def __init__(self, data_root, agent_id, transform=None):
        super().__init__(data_root)
        self.agent_path = os.path.join(data_root, agent_id)
        self.image_pairs = self._find_image_pairs()
        self.transform = transform

    def _find_image_pairs(self):
        files = sorted(os.listdir(self.agent_path))
        image_pairs = []
        for f in files:
            if "_camera0.png" in f:
                base = f.replace("_camera0.png", "")
                if f"{base}_camera2.png" in files:
                    image_pairs.append((f, f"{base}_camera2.png"))
        return image_pairs

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, idx):
        cam0, cam2 = self.image_pairs[idx]
        img0 = os.path.join(self.agent_path, cam0)
        img2 = os.path.join(self.agent_path, cam2)
        return {
            "front_image": img0,
            "rear_image": img2,
        }

In [ ]:
import torch
from vggt.models.vggt import VGGT
from vggt.utils.load_fn import load_and_preprocess_images
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
# bfloat16 is supported on Ampere GPUs (Compute Capability 8.0+) 
dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16

# Initialize the model and load the pretrained weights.
# This will automatically download the model weights the first time it's run, which may take a while.
model = VGGT.from_pretrained("facebook/VGGT-1B").to(device)

dataset = OPV2VImageDataset(data_root="/data/opv2v/2021_08_16_22_26_54", agent_id="641", transform=None)
train_loader = DataLoader(dataset, batch_size=50, shuffle=True)
# load the first batch of images from the dataset
for batch in train_loader:
    img_batch = batch['front_image'] + batch['rear_image']
    images = load_and_preprocess_images(img_batch).to(device)

    with torch.no_grad():
        with torch.cuda.amp.autocast(dtype=dtype):
            # Predict attributes including cameras, depth maps, and point maps.
            predictions = model(images)
    print("Done once")

/users/bangya/miniconda3/envs/vggt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OutOfMemoryError: CUDA out of memory. Tried to allocate 408.00 MiB. GPU 

: 